In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist
import time

In [2]:
X_df = pd.read_csv('data/data.csv', header=None)
y_df = pd.read_csv('data/label.csv', header=None)

In [3]:
def euclidean_dist(X, Y):
    return cdist(X, Y, 'euclidean')

def cosine_dist(X, Y):
    return cdist(X, Y, 'cosine')

def generalized_jaccard_dist(X, Y):
    min_sum = np.sum(np.minimum(X[:, np.newaxis], Y[np.newaxis, :]), axis=2)
    max_sum = np.sum(np.maximum(X[:, np.newaxis], Y[np.newaxis, :]), axis=2)
    jaccard_sim = min_sum / max_sum
    return 1 - jaccard_sim

In [4]:
def k_means(data, k, dist_fn, max_iter=500):
    centroids = data.sample(n=k).values
    last_sse = float('inf')
    for _ in range(max_iter):
        distances = dist_fn(data.values, centroids)
        clusters = np.argmin(distances, axis=1)
        sse = np.sum(np.min(distances, axis=1)**2)
        if sse >= last_sse:
            break
        last_sse = sse
        new_centroids = np.array([data.values[clusters == j].mean(axis=0) for j in range(k)])
        if np.all(centroids == new_centroids):
            break
        centroids = new_centroids
    return clusters, centroids, last_sse

In [5]:
k = 10
clusters_euclidean, _, sse_euclidean = k_means(X_df, k, euclidean_dist)
clusters_cosine, _, sse_cosine = k_means(X_df, k, cosine_dist)
clusters_jaccard, _, sse_jaccard = k_means(X_df, k, generalized_jaccard_dist)
print(f"Sum of Squared Error for Euclidean: {sse_euclidean:.2f}")
print(f"Sum of Squared Error for Cosine: {sse_cosine:.2f}")
print(f"Sum of Squared Error for Generalized Jaccard: {sse_jaccard:.2f}")
best_method = min([('Euclidean', sse_euclidean), ('Cosine', sse_cosine), ('Jaccard', sse_jaccard)], key=lambda x: x[1])
print(f"The most optimum algorithm based on SSE is: {best_method[0]} with an SSE of {best_method[1]:.2f}")


Sum of Squared Error for Euclidean: 25683570207.98
Sum of Squared Error for Cosine: 682.26
Sum of Squared Error for Generalized Jaccard: 4164.18
The most optimum algorithm based on SSE is: Cosine with an SSE of 682.26


In [6]:
def assign_clusters(clusters, gold):
    cluster_labels = {}
    for cluster in set(clusters):
        labels_in_cluster = gold[clusters == cluster]
        most_common = np.bincount(labels_in_cluster).argmax()
        cluster_labels[cluster] = most_common
    return cluster_labels

def get_cluster_predictions(clusters, cluster_labels):
    return np.array([cluster_labels[cluster] for cluster in clusters])


In [7]:
def calculate_accuracy(predicted_labels, gold):
    return np.mean(predicted_labels == gold)

# Assuming you have the cluster results from previous steps
gold = y_df.values.flatten()

# Assign labels to each cluster
euc_cluster = assign_clusters(clusters_euclidean, gold)
cos_cluster = assign_clusters(clusters_cosine, gold)
jac_cluster = assign_clusters(clusters_jaccard, gold)

# Get the predicted labels for each data point
preds_eucl = get_cluster_predictions(clusters_euclidean, euc_cluster)
preds_cos = get_cluster_predictions(clusters_cosine, cos_cluster)
preds_jac = get_cluster_predictions(clusters_jaccard, jac_cluster)

# Calculate accuracies
acc_euc = calculate_accuracy(preds_eucl, gold)
acc_cos = calculate_accuracy(preds_cos, gold)
acc_jac = calculate_accuracy(preds_jac, gold)


In [8]:
print(f"Accuracy using Euclidean Distance: {acc_euc:.2f}")
print(f"Accuracy using Cosine Similarity: {acc_cos:.2f}")
print(f"Accuracy using Jaccard: {acc_jac:.2f}")

best_method = max([('Euclidean', acc_euc), ('Cosine', acc_cos), ('Jaccard', acc_jac)], key=lambda x: x[1])
print(f"The best method based on accuracy is: {best_method[0]} with an accuracy of {best_method[1]:.2f}")


Accuracy using Euclidean Distance: 0.58
Accuracy using Cosine Similarity: 0.61
Accuracy using Jaccard: 0.50
The best method based on accuracy is: Cosine with an accuracy of 0.61


In [9]:
def k_means(data, k, distance_function, max_iter=500):
    centroids = data.sample(n=k).values
    last_sse = float('inf')
    iterations = 0
    start_time = time.time()
    for _ in range(max_iter):
        distances = distance_function(data.values, centroids)
        clusters = np.argmin(distances, axis=1)
        sse = np.sum(np.min(distances, axis=1)**2)
        new_centroids = np.array([data.values[clusters == j].mean(axis=0) for j in range(k)])
        if np.all(centroids == new_centroids) or sse > last_sse:
            break
        last_sse = sse
        centroids = new_centroids
        iterations += 1
    end_time = time.time()
    return clusters, centroids, iterations, end_time - start_time

In [10]:
_, _, iters_euc, time_euclidean = k_means(X_df, k, euclidean_dist)
_, _, iters_cos, time_cosine = k_means(X_df, k, cosine_dist)
_, _, iters_jac, time_jaccard = k_means(X_df, k, generalized_jaccard_dist)
print(f"Euclidean: {iters_euc} iterations, {time_euclidean:.2f} seconds")
print(f"Cosine: {iters_cos} iterations, {time_cosine:.2f} seconds")
print(f"Generalized Jaccard: {iters_jac} iterations, {time_jaccard:.2f} seconds")
most_iterations = max([('Euclidean', iters_euc), ('Cosine', iters_cos), ('Jaccard', iters_jac)], key=lambda x: x[1])
longest_time = max([('Euclidean', time_euclidean), ('Cosine', time_cosine), ('Jaccard', time_jaccard)], key=lambda x: x[1])
print(f"Max iterations required for: {most_iterations[0]} with {most_iterations[1]} iterations")
print(f"Most time required for: {longest_time[0]} with {longest_time[1]:.2f} seconds")

Euclidean: 92 iterations, 18.11 seconds
Cosine: 61 iterations, 15.92 seconds
Generalized Jaccard: 1 iterations, 2.55 seconds
Max iterations required for: Euclidean with 92 iterations
Most time required for: Euclidean with 18.11 seconds


In [12]:
def initialize_centroids(data, k):
    return data.sample(n=k).values

def assign_clusters(data, centroids, distance_function):
    distances = distance_function(data.values, centroids)
    return np.argmin(distances, axis=1)

def update_centroids(data, clusters, k):
    return np.array([data.values[clusters == j].mean(axis=0) for j in range(k)])

def calculate_sse(data, clusters, centroids):
    distances = np.sqrt(((data.values - centroids[clusters])**2).sum(axis=1))
    return np.sum(distances**2)

def centroids_not_changed(old_centroids, new_centroids):
    return np.allclose(old_centroids, new_centroids, atol=1e-6)

def sse_increased(sse_history):
    return len(sse_history) > 1 and sse_history[-1] > sse_history[-2]

def k_means_sse_tracking(data, k, distance_function, max_iter=100):
    centroids = initialize_centroids(data, k)
    sse_history = []
    for iteration in range(max_iter):
        clusters = assign_clusters(data, centroids, distance_function)
        new_centroids = update_centroids(data, clusters, k)
        sse = calculate_sse(data, clusters, centroids)
        sse_history.append(sse)

        if centroids_not_changed(centroids, new_centroids) or sse_increased(sse_history):
            break
        centroids = new_centroids
    return centroids, clusters, sse_history

In [13]:
k = 10 
centroids_euclidean, clusters_euclidean, sse_history_euclidean = k_means_sse_tracking(X_df, k, euclidean_dist)
centroids_cosine, clusters_cosine, sse_history_cosine = k_means_sse_tracking(X_df, k, cosine_dist)
centroids_jaccard, clusters_jaccard, sse_history_jaccard = k_means_sse_tracking(X_df, k, generalized_jaccard_dist)
print(f"Sum of Square Error using Euclidean: {sse_history_euclidean[-1]:.2f}")
print(f"Sum of Square Error using Cosine: {sse_history_cosine[-1]:.2f}")
print(f"Sum of Squared Error using Jaccard: {sse_history_jaccard[-1]:.2f}")

Sum of Square Error using Euclidean: 25472003422.71
Sum of Square Error using Cosine: 25617574227.59
Sum of Squared Error using Jaccard: 25446117809.36
